---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

df_train     = pd.read_csv('train.csv', encoding='ISO-8859-1')
df_test_real_world      = pd.read_csv('test.csv', encoding='ISO-8859-1')
df_addresses = pd.read_csv('addresses.csv', encoding='ISO-8859-1')
print(np.shape(df_train))
print(np.shape(df_test_real_world))
print(np.shape(df_addresses))

C:\Users\Spencer\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(250306, 34)
(61001, 27)
(311307, 2)


In [2]:
df_train = pd.merge(df_train, df_addresses, on='ticket_id', how='inner')
df_test_real_world = pd.merge(df_test_real_world, df_addresses, on='ticket_id', how='inner')

# just guesses based on column names as to what will be most significant.
# also, did not keep columns which are not in both train.csv and test_real_world.csv
var_cols = \
    [ \
     'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', \
     'clean_up_cost', 'judgment_amount', \
     'zip_code', 'violation_code' \
    ]

X_train = df_train[var_cols]
X_test_real_world  = df_test_real_world[var_cols]

# objects to categories - print X_train.dtypes. For those which are objects, change to category.

X_train['zip_code']       = X_train['zip_code'].astype('category')
X_train['violation_code'] = X_train['violation_code'].astype('category')
X_test_real_world['zip_code']        = X_test_real_world['zip_code'].astype('category')
X_test_real_world['violation_code']  = X_test_real_world['violation_code'].astype('category')
y_train = df_train['compliance'].astype('category')

# remove empty compliance values from training set
X_train = X_train[~y_train.isnull()]
y_train = y_train[~y_train.isnull()]

# change categories further to integer category codes.
# Not sure what is going on exactly, but still issues with converting stings to floats for above categorical columns.
# Code below fixes that. Grabbed from stack overflow somewhere.

X_train[['zip_code', 'violation_code']] = X_train[['zip_code', 'violation_code']].apply(lambda x: x.cat.codes)
X_test_real_world[['zip_code', 'violation_code']] = X_test_real_world[['zip_code', 'violation_code']].apply(lambda x: x.cat.codes)
# y_train = y_train.astype('int')

C:\Users\Spencer\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Spencer\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Spencer\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#

In [4]:
df_test_real_world.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,address
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,"10041 roseberry, Detroit MI"
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,"18520 evergreen, Detroit MI"
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,"18520 evergreen, Detroit MI"
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN,"1835 central, Detroit MI"
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN,"1700 central, Detroit MI"


In [5]:
from sklearn.naive_bayes import GaussianNB
from adspy_shared_utilities import plot_class_regions_for_classifier
from sklearn.model_selection import train_test_split

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, random_state=0)

nbclf = GaussianNB().fit(X_train_2, y_train_2)

In [7]:
print(nbclf.score(X_train_2, y_train_2))
print(nbclf.score(X_test_2, y_test_2))

0.934259027604
0.933750312735


In [8]:
nbclf = GaussianNB().fit(X_train, y_train)
probs = nbclf.predict_proba(X_test_real_world)[:, 1]

In [9]:
np.shape(probs)

(61001,)

In [13]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from adspy_shared_utilities import plot_class_regions_for_classifier
from sklearn.model_selection import train_test_split

def blight_model():
    
    df_train           = pd.read_csv('train.csv', encoding='ISO-8859-1')
    df_test_real_world = pd.read_csv('test.csv', encoding='ISO-8859-1')
    df_addresses       = pd.read_csv('addresses.csv', encoding='ISO-8859-1')
    
    df_train = pd.merge(df_train, df_addresses, on='ticket_id', how='inner')
    df_test_real_world = pd.merge(df_test_real_world, df_addresses, on='ticket_id', how='inner')

    # just guesses based on column names as to what will be most significant.
    # also, did not keep columns which are not in both train.csv and test_real_world.csv
    var_cols = \
        [ \
         'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', \
         'clean_up_cost', 'judgment_amount', \
         'zip_code', 'violation_code' \
        ]

    X_train = df_train[var_cols]
    X_test_real_world  = df_test_real_world[var_cols]

    # objects to categories - print X_train.dtypes. For those which are objects, change to category.

    X_train['zip_code']       = X_train['zip_code'].astype('category')
    X_train['violation_code'] = X_train['violation_code'].astype('category')
    X_test_real_world['zip_code']        = X_test_real_world['zip_code'].astype('category')
    X_test_real_world['violation_code']  = X_test_real_world['violation_code'].astype('category')
    y_train = df_train['compliance'].astype('category')

    # remove empty compliance values from training set
    X_train = X_train[~y_train.isnull()]
    y_train = y_train[~y_train.isnull()]

    # change categories further to integer category codes.
    # Not sure what is going on exactly, but still issues with converting stings to floats for above categorical columns.
    # Code below fixes that. Grabbed from stack overflow somewhere.

    X_train[['zip_code', 'violation_code']] = X_train[['zip_code', 'violation_code']].apply(lambda x: x.cat.codes)
    X_test_real_world[['zip_code', 'violation_code']] = X_test_real_world[['zip_code', 'violation_code']].apply(lambda x: x.cat.codes)
    # y_train = y_train.astype('int')
    
    nbclf = GaussianNB().fit(X_train, y_train)
    probabilities = nbclf.predict_proba(X_test_real_world)[:, 1]
    probabilities = pd.Series(probabilities, index=X_test_real_world.index)
    
    return probabilities


In [ ]:
blight_model()

In [ ]:
import numpy as np
import pandas as pd

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

def blight_model():
    
    df_train           = pd.read_csv('train.csv', encoding='ISO-8859-1')
    df_test_real_world = pd.read_csv('test.csv', encoding='ISO-8859-1')
    df_addresses       = pd.read_csv('addresses.csv', encoding='ISO-8859-1')
    
    df_train = pd.merge(df_train, df_addresses, on='ticket_id', how='inner')
    df_test_real_world = pd.merge(df_test_real_world, df_addresses, on='ticket_id', how='inner')

    # just guesses based on column names as to what will be most significant.
    # also, did not keep columns which are not in both train.csv and test_real_world.csv
    var_cols = \
        [ \
         'fine_amount', 'admin_fee', 'state_fee', 'late_fee', 'discount_amount', \
         'clean_up_cost', 'judgment_amount', \
         'zip_code', 'violation_code' \
        ]

    X_train = df_train[var_cols]
    X_test_real_world  = df_test_real_world[var_cols]

    # objects to categories - print X_train.dtypes. For those which are objects, change to category.

    X_train['zip_code']       = X_train['zip_code'].astype('category')
    X_train['violation_code'] = X_train['violation_code'].astype('category')
    X_test_real_world['zip_code']        = X_test_real_world['zip_code'].astype('category')
    X_test_real_world['violation_code']  = X_test_real_world['violation_code'].astype('category')
    y_train = df_train['compliance'].astype('category')

    # remove empty compliance values from training set
    X_train = X_train[~y_train.isnull()]
    y_train = y_train[~y_train.isnull()]

    # change categories further to integer category codes.
    # Not sure what is going on exactly, but still issues with converting stings to floats for above categorical columns.
    # Code below fixes that. Grabbed from stack overflow somewhere.

    X_train[['zip_code', 'violation_code']] = X_train[['zip_code', 'violation_code']].apply(lambda x: x.cat.codes)
    X_test_real_world[['zip_code', 'violation_code']] = X_test_real_world[['zip_code', 'violation_code']].apply(lambda x: x.cat.codes)
    # y_train = y_train.astype('int')
    
    # all above was data organization. Now the actual ML.
    # nbclf = GaussianNB().fit(X_train, y_train)
    # probabilities = nbclf.predict_proba(X_test_real_world)[:, 1]
    grid_vals = {'learning_rate': [0.05, 0.5], 'max_depth': [3, 4]}
    clf = GradientBoostingClassifier(learning_rate = 0.1, max_depth = 4, random_state=0)
    # grid_clf = GridSearchCV(clf, param_grid=grid_vals, scoring='roc_auc')
    clf.fit(X_train, y_train)
    probabilities = clf.predict_proba(X_test_real_world)[:, 1]
    # probabilities = grid_clf.predict_proba(X_test_real_world)[:, 1]
    
    # and now arrange results into assignment's requested output.
    probabilities = pd.Series(probabilities, index=df_test_real_world.ticket_id)
    
    return probabilities


In [ ]:
blight_model()